# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [51]:
# your code goes here
import pandas as pd
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     10910 non-null  int64  
 1   Customer                       10910 non-null  object 
 2   State                          10279 non-null  object 
 3   Customer Lifetime Value        10910 non-null  float64
 4   Response                       10279 non-null  object 
 5   Coverage                       10910 non-null  object 
 6   Education                      10910 non-null  object 
 7   Effective To Date              10910 non-null  object 
 8   EmploymentStatus               10910 non-null  object 
 9   Gender                         10910 non-null  object 
 10  Income                         10910 non-null  int64  
 11  Location Code                  10910 non-null  object 
 12  Marital Status                 10910 non-null 

In [ ]:
#1
df["High Claims"]= df.apply(lambda row: 1 if row["Total Claim Amount"] >=1000 and row["Response"] == "Yes" else 0, axis=1)
df_high_claims = df.groupby("High Claims")
df_high_claims.get_group(1)


In [ ]:
#2
df_yes_response = df.groupby("Response")
df_yes_response.get_group("Yes").pivot_table(index="Gender", columns="Policy Type", values=["Total Claim Amount"])


Total Claim Amount                           
Policy Type     Corporate Auto Personal Auto Special Auto
Gender                                                   
F                   433.738499    452.965929   453.280164
M                   408.582459    457.010178   429.527942

In [ ]:
#3
df_customer_policy_state = df.pivot_table(index="Policy", columns="State", values=["Customer"], aggfunc="count")
df_customer_policy_state_filtered = df_customer_policy_state[df_customer_policy_state>500]
df_customer_policy_state_filtered


Customer                                     
State         Arizona California Nevada  Oregon Washington
Policy                                                    
Corporate L1      NaN        NaN    NaN     NaN        NaN
Corporate L2      NaN        NaN    NaN     NaN        NaN
Corporate L3      NaN        NaN    NaN     NaN        NaN
Personal L1       NaN        NaN    NaN     NaN        NaN
Personal L2       NaN      905.0    NaN   618.0        NaN
Personal L3     742.0     1250.0    NaN  1157.0        NaN
Special L1        NaN        NaN    NaN     NaN        NaN
Special L2        NaN        NaN    NaN     NaN        NaN
Special L3        NaN        NaN    NaN     NaN        NaN

In [46]:
#4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

df4 = df.pivot_table(index="Gender", columns="Education", values=["Customer Lifetime Value"], aggfunc=["min", "max", "median"])
df4 = df4.round(2)
df4.head()

min                                         \
          Customer Lifetime Value                                          
Education                Bachelor  College   Doctor High School or Below   
Gender                                                                     
F                         1904.00  1898.68  2395.57              2144.92   
M                         1898.01  1918.12  2267.60              1940.98   

                                       max                      \
                   Customer Lifetime Value                       
Education   Master                Bachelor   College    Doctor   
Gender                                                           
F          2417.78                73225.96  61850.19  44856.11   
M          2272.31                67907.27  61134.68  32677.34   

                                                          median           \
                                         Customer Lifetime Value            
Education High School or Below    Master                Bachelor  College   
Gender                                                                      
F                     55277.45  51016.07                 5640.51  5623.61   
M                     83325.38  50568.26                 5548.03  6005.85   

                                                  
                                                  
Education   Doctor High School or Below   Master  
Gender                                            
F          5332.46              6039.55  5729.86  
M          5577.67              6286.73  5579.10

In [53]:
#5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

df5 = df.pivot_table(index="State", columns="Months Since Policy Inception", values=["Number of Policies"], aggfunc="sum")
df5.head()

Number of Policies                              \
Months Since Policy Inception                 0    1    2    3    4   5   6    
State                                                                          
Arizona                                       53   44   36   67   43  54  70   
California                                   189  107   70  114  143  80  92   
Nevada                                        18   14   24   24   24  14   6   
Oregon                                        54   99  123   91   67  88  73   
Washington                                    25   25   39   29   16  15  11   

                                            ...                               \
Months Since Policy Inception  7   8    9   ...  90   91  92  93  94  95  96   
State                                       ...                                
Arizona                        27  61   44  ...  40   55  47  53  92  33  17   
California                     87  93  143  ...  87   50  60  63  70  63  76   
Nevada                         32  51   37  ...  34   27   2  43   9  38  12   
Oregon                         88  83   95  ...  54  119  97  82  54  57  62   
Washington                     17  21    5  ...  55   18  20  26  16  23  19   

                                            
Months Since Policy Inception  97  98   99  
State                                       
Arizona                        18  34   70  
California                     50  58  117  
Nevada                         21  20   15  
Oregon                         67  50   59  
Washington                      5  28   19  

[5 rows x 100 columns]

In [ ]:
#6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.